In [5]:
from textwrap import dedent
import numpy as np
from scipy import interp
from scipy.stats import lognorm
from scipy.integrate import fixed_quad
from quantecon import compute_fixed_point

class LucasTree(object):
    """
    class to solve for the price of a lucas tree in the lucas asset pricing  model
    Parameters
    ---------
    gamma: scalar(float)
        the coefficient of risk aversion in the household's crra utility function
    beta: scalar(float)
        the household's discount factor
    alpha: scalar(float)
        the correlation coefficient in the shock process
    sigma: scalar(float)
        the volatility of the shock process
    grid: array_like(float), optional(default=None)
        the grid point on which to evaluate the asset prices, grid
        points should be nonnegative. if none is passed, we will create
        a reasonable one for you
        
    attributes
    ----------
    gamma, beta, alpha, sigma, grid: see parameters
    grid_min, grid_max, grid_size: scalar(int)
        properties for grid upon which prices are evaluated
    phi: scipy.stats.lognorm
        the distribution for the shock process
        
    examples
    ---------
    >>>tree=LucasTree(gamma=2, beta=0.95, alpha=0.90, sigma=0.1)
    >>>grid, price_vals=tree.gird, tree.compute_lt_price()
    
    """
    
    def __init__(self, gamma, beta, alpha, sigma, grid=None):
        self.gamma=gamma
        self.beta=beta
        self.alpha=alpha
        self.sigma=sigma
        
        #==set up grid==#
        if grid is None:
            (self.grid, self.grid_min,
            self.grid_max, self.grid_size)=self._new_grid()
        else:
            self.grid=np.asarray(grid)
            self.grid_min=min(grid)
            self.grid_max=max(grid)
            self.grid_size=len(grid)
            
        #==set up distribution for shocks==#
        self.phi=lognorm(sigma)
        
        #==set up integration bounds.4 standard deviations. make them
        #private attribute b/c users don't need to see them, but we
        #only want to compute them once.==#
        self._int_min=np.exp(-4.0*sigma)
        self._int_max=np.exp(4.0*sigma)
        
        #==set up h from the Lucas Operator==#
        
        def __repr__(self):
            m="LucasTree(gamma={g}, beta={b}, alpha={a}, sigma={s})"
            return m.format(g=self.gamma, b=self.beta, a=self.alpha, s=self.sigma)
        
        def __str__(self):
            m="""\
            Lucas Pricing Model(Lucas, 1978):
            -gamma(coefficient of risk aversion):              {g}
            -beta(discount parameter):                             {b}
            -alpha(correlation coefficient in shock process): {a}
            -sigma(volatility of shock process):                  {s}
            -grid bounds(bounds for where to compute prices): ({gl:g}, {gu, g})
            -grid points(number of grid points):              {gs}
            """
            return dedent(m.format(g=self.gamma, b=self.beta, a=self.alpha,
                                   s=self.sigma, gl=self.grid_min,
                                   gu=self.grid_max, gs=self.grid_size))
        
        def _init_h(self):
            """
            compute the function h in the lucas operator as a vector of
            values on the grid
            
            recall that h(y)=beta*int u'(G(y,z)) G(y,z) phi(dz)
            """
            alpha, gamma, beta=self.alpha, self.gamma, self.beta
            grid, grid_size=self.grid, self.grid_size
            
            h=np.empty(grid_size)
            
            for i, y in enumerate(grid):
                #==u'(G(y, z))G(y,z)==#
                integrand=lambda z: (y**alpha*z)**(1-gamma)
                h[i]=beta*self.integrate(integrand)
                
            return h
        
        def _new_grid(self):
            """
            construct the default grid for the problem
            
            this is defined to be np.linspace(0, 10, 100) when alpha>1
            and 100 evenly spaced points covering 4 standard deviations
            when alpha<1
            """
            grid_size=100
            if abs(self.alpha)>=1.0:
                grid_min, grid_max=0.0, 10.0
            else:
                #==set the grid interval to contain most of the mass of the
                # stationary distribution of the consumption endowment==#
                ssd=self.sigma/np.sqrt(1-self.alpha**2)
                grid_min, grid_max=np.exp(-4*ssd), np.exp(4*ssd)
                
            grid=np.linspace(grid_min, grid_max, grid_size)
            
            return grid, grid_min, grid_max, grid_size
        
        def integrate(self, g, int_min=None, int_max=None):
            """
            integrate the function g(z)*self.phi(z) from int_min to int_max
            
            parameters
            ----------
            g: function
                the function which to integrate
                
            int_min, int_max: scalar(float), optional
                the bounds of integration. if either of these parameters are
                None (the default), they will be set to 4 standard
                deviations above and below the mean.
                
            returns
            ----------
            result: scalar(float)
                the result of the integration
                
            """
            #==simplify notation==#
            phi=self.phi
            if int_min is None:
                int_min=self.int_min
            if int_max is None:
                int_max=self.int_max
                
            #==set up integrand and integrate==#
            integrand=lambda z: g(z)*phi.pdf(z)
            result, error=fixed_quad(integrand, int_min, int_max)
            return result
        
        def lucas_operator(self, f, Tf=None):
            """
            the approximate Lucas operator, which computes and returns the
            updated function Tf on the grid points.
            
            parameters
            ---------
            f: array_like(float)
                a candidate function on R_+ represented as points on a grid
                and should be flat NumPy array with len(f)=len(grid)
                
            Tf: array_like(float)
                optional storage array for Tf
                
            returns
            ---------
            Tf: array_like(float)
                the updated function Tf
            
            notes
            ------
            the argument 'Tf' is optional, but recommended. if it is passed
            into this function, the we do not have to allocate any memory
            for the array here as this function is often called many times
            in an iterative algorithm, this can save significant computation
            time.
            
            """
            grid, h=self.grid, self.h
            alpha, beta=self.alpha, self.beta
            
            #==set up storage if needed==#
            if Tf is None:
                Tf=np.empty_like(f)
                
            #==apply the T operator to f==#
            Af=lambda x: interp(x, grid, f) #piecewise linear interpolation
            
            for i, y in enumerate(grid):
                Tf[i]=h[i]+beta*self.integrate(lambda z: Af(y**alpha*z))
                
            return Tf
        
        def compute_lt_price(self, error_tol=1e-3, max_iter=50, verbose=0):
            """
            compute the equilibrium price function associated with lucas tree lt
            
            parameters
            ----------
            error_tol, max_iter, verbose
                arguments to be passed directly to
                quantecon.compute_fixed_point. see that docstring for more
                information
                
            returns
            ----------
            price: array_like(float)
                the prices at the grid points in the attribute 'grid' of the 
                object
                
            """
            #==simplify notation==#
            grid, grid_size=self.grid, self.grid_size
            lucas_operator, gamma=self.lucas_operator, self.gamma
            
            #==create storage array for compute_fixed_point. reduces memory
            #allocation and speeds code up==#
            Tf=np.empty(grid_size)
            
            #==initial guess, just a vector of zeros==#
            f_init=np.zeros(grid_size)
            f=compute_fixed_point(lucas_operator, f_init, error_tol,
                                  max_iter, verbose, Tf=Tf)
            
            price=f*grid*gamma
            
            return price
            